In [1]:
 #!pip install librosa


In [2]:
"""import librosa as lib
import os
import pandas as pd
import pylab
import numpy as np
import librosa.display
import glob """

'import librosa as lib\nimport os\nimport pandas as pd\nimport pylab\nimport numpy as np\nimport librosa.display\nimport glob '

In [3]:
"""def convert(filename):   
    sig, fs = lib.load(filename)   
    # make pictures name 
    save_path = filename+'.jpg'

    pylab.axis('off') # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    S = librosa.feature.melspectrogram(y=sig, sr=fs)
    lib.display.specshow(librosa.power_to_db(S, ref=np.max))
    pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
    pylab.close()"""

"def convert(filename):   \n    sig, fs = lib.load(filename)   \n    # make pictures name \n    save_path = filename+'.jpg'\n\n    pylab.axis('off') # no axis\n    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge\n    S = librosa.feature.melspectrogram(y=sig, sr=fs)\n    lib.display.specshow(librosa.power_to_db(S, ref=np.max))\n    pylab.savefig(save_path, bbox_inches=None, pad_inches=0)\n    pylab.close()"

In [4]:
##"""for i in range(11):
#    for filename in glob.glob(r'D:\Projects\urban_sound\UrbanSound8K\audio\fold'+str(i)+'\*.wav'):
 #       convert(filename)"""

In [1]:
import pandas as pd
data = pd.read_csv(r'D:\Projects\urban_sound\UrbanSound8K\metadata\UrbanSound8K.csv')

In [2]:
print(data.head())

      slice_file_name    fsID  start        end  salience  fold  classID  \
0    100032-3-0-0.wav  100032    0.0   0.317551         1     5        3   
1  100263-2-0-117.wav  100263   58.5  62.500000         1     5        2   
2  100263-2-0-121.wav  100263   60.5  64.500000         1     5        2   
3  100263-2-0-126.wav  100263   63.0  67.000000         1     5        2   
4  100263-2-0-137.wav  100263   68.5  72.500000         1     5        2   

              class  
0          dog_bark  
1  children_playing  
2  children_playing  
3  children_playing  
4  children_playing  


In [3]:
test = data.loc[data['fold'] == 6]
train = data.loc[data['fold'] != 6]

In [4]:
#cv2.imshow('in',cv2.imread(r'D:\Projects\urban_sound\UrbanSound8K\audio\fold7\99812-1-6-0.wav.jpg'))


In [6]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import cv2

loc = r'D:\Projects\urban_sound\UrbanSound8K\audio'
img_lst = np.array(train.slice_file_name.tolist())
folds = np.array(train.fold)
print(img_lst.shape)
img_lst = [loc +'\\'+'fold'+str(fold)+'\\'+ s + '.jpg' for s,fold in zip(img_lst,folds)]
X = []
for s in img_lst:
    X.append(cv2.resize(cv2.imread(s, cv2.IMREAD_COLOR),(150,150),interpolation=cv2.INTER_CUBIC))
#X = [cv2.imshow('img',s) for s in X]

y = np.array(train.classID.tolist())

(7909,)


In [7]:
import gc
del train
gc.collect()

0

In [8]:
X = np.array(X)
#print(X[0])
print(y)

[3 2 2 ... 1 1 1]


In [9]:
X = X.astype('float')

In [10]:
X /= 255.0 

In [11]:
X.shape

(7909, 150, 150, 3)

In [12]:
# construct the image generator for data augmentation
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator()

Using TensorFlow backend.


In [13]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
lb = LabelEncoder()
y = np_utils.to_categorical(lb.fit_transform(y))

In [14]:
import keras
#base = keras.applications.nasnet.NASNetLarge(input_shape=None, include_top=False, weights=None, input_tensor=None, pooling=None, classes=10)
base = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=10)

In [15]:
test_lst = np.array(test.slice_file_name.tolist())
folds = np.array(test.fold)
test_lst = [loc +'\\'+'fold'+str(fold)+'\\'+ s + '.jpg' for s,fold in zip(test_lst,folds)]
#print(test_lst)
test_X = []
for s in test_lst:
    test_X.append(cv2.resize(cv2.imread(s, cv2.IMREAD_COLOR),(150,150),interpolation=cv2.INTER_CUBIC))
#X = [cv2.imshow('img',s) for s in X]
test_X = np.array(test_X)
test_X = test_X.astype('float')
test_X /= 255.0 
test_y = np.array(test.classID.tolist())
test_y = np_utils.to_categorical(lb.fit_transform(test_y))
print(test_X.shape)

(823, 150, 150, 3)


In [20]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD

BS = 48
EPOCHS = 25
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base.input, outputs=predictions)
for layer in base.layers:
    layer.trainable = True
model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['categorical_accuracy','top_k_categorical_accuracy'])
model.fit_generator(aug.flow(X, y, batch_size=BS),
	steps_per_epoch=7895 // BS,
	epochs=EPOCHS, verbose=1)

Epoch 1/25


ResourceExhaustedError: OOM when allocating tensor with shape[48,160,7,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_2/SGD/gradients/AddN_175-0-TransposeNHWCToNCHW-LayoutOptimizer}} = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_2/SGD/gradients/batch_normalization_127/cond/FusedBatchNorm/Switch_grad/cond_grad, PermConstNHWCToNCHW-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node dense_6/Softmax/_14177}} = _HostRecv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_29150_dense_6/Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a lis

In [17]:
model.save(r'D:\Projects\urban_sound\model\model_6.h5')

In [ ]:
model.evaluate(x=test_X, y=test_y, batch_size=None, verbose=1)